<a href="https://colab.research.google.com/github/aaaksenova/NLP_ABSA_project/blob/change/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
# Отмечали в этом ресторане день рождение
# O        O O    B-Whole   O    O

In [3]:
! pip install stanza
! pip install -U 'scikit-learn<0.24'

     |████████████████████████████████| 432 kB 4.4 MB/s 
     |████████████████████████████████| 170 kB 43.4 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=08b18ba73bdd19cdbb5c1bec791e328e9727cba9941ca7d0bd967759afc8823d
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji
     |████████████████████████████████| 6.8 MB 4.3 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [4]:
import stanza

In [5]:
stanza.download('ru')

2021-12-24 07:19:16 INFO: Downloading default packages for language: ru (Russian)...


2021-12-24 07:19:53 INFO: Finished downloading models and saved to /root/stanza_resources.


In [6]:
nlp = stanza.Pipeline('ru', processors='tokenize,pos')

2021-12-24 07:19:53 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2021-12-24 07:19:53 INFO: Use device: cpu
2021-12-24 07:19:53 INFO: Loading: tokenize
2021-12-24 07:19:53 INFO: Loading: pos
2021-12-24 07:19:54 INFO: Done loading processors!


In [7]:
train_asp = pd.read_csv(
    'train_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)
train_texts = pd.read_csv('train_reviews.txt', delimiter='\t', names=['text_id','text'])

In [8]:
# process text https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2
nlp(train_texts['text'][0])

[
  [
    {
      "id": 1,
      "text": "День",
      "upos": "NOUN",
      "feats": "Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing",
      "start_char": 0,
      "end_char": 4
    },
    {
      "id": 2,
      "text": "8-го",
      "upos": "ADJ",
      "feats": "Case=Gen|Degree=Pos|Gender=Masc|Number=Sing",
      "start_char": 5,
      "end_char": 9
    },
    {
      "id": 3,
      "text": "марта",
      "upos": "NOUN",
      "feats": "Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing",
      "start_char": 10,
      "end_char": 15
    },
    {
      "id": 4,
      "text": "прошёл",
      "upos": "VERB",
      "feats": "Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act",
      "start_char": 16,
      "end_char": 22
    },
    {
      "id": 5,
      "text": ",",
      "upos": "PUNCT",
      "start_char": 22,
      "end_char": 23
    },
    {
      "id": 6,
      "text": "можно",
      "upos": "ADV",
      "feats": "Degree=Pos",
      "start_char": 24,
      "e

In [9]:
reviews = {}
with open('train_reviews.txt') as f:
  for line in f:
    line = line.rstrip('\r\n').split('\t')
    reviews[line[0]] = line[1]

In [10]:
from collections import defaultdict
aspects = defaultdict(list)

In [11]:
with open('train_aspects.txt') as f:
  for line in f:
    line = line.rstrip('\r\n').split('\t')
    keys = ('category', 'mention', 'start', 'end', 'sentiment')
    # ['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
    # тут можно отдельно запомнить начало и конец каждого упоминания
    aspects[line[0]].append(dict(zip(keys, line[1:])))

In [12]:
from tqdm.auto import tqdm

In [17]:
# здесь код для упоминаний из 1 токена
# ВАЖНО: для более длинных упоминаний нужно доделать ещё немного
texts = []
bios = []
poses = []
sentences = []
sent_id=0
for text_id, text in tqdm(reviews.items()):
  processed = nlp(text)
  text_data = []
  bio_data = []
  pos_data = []
  count_token = 0
  for token in processed.iter_tokens():
    add = False
    for mention in aspects[text_id]:
      if token.start_char == int(mention['start']) and token.end_char <= int(mention['end']):
        text_data.append(token.text)
        bio_data.append('B-'+mention['category'])
        pos_data.append(token.words[0].pos)
        count_token += 1
        add = True
      elif token.start_char > int(mention['start']) and token.end_char <= int(mention['end']):
        text_data.append(token.text)
        bio_data.append('I-'+mention['category'])
        pos_data.append(token.words[0].pos)
        count_token += 1
        add = True
    if not add:
      text_data.append(token.text)
      bio_data.append('O')
      pos_data.append(token.words[0].pos)
      count_token += 1
  texts.extend(text_data)
  bios.extend(bio_data)
  poses.extend(pos_data)
  sentences.extend([f'Sentence: {sent_id}'] + ['']*(count_token - 1))
  sent_id+=1

  0%|          | 0/284 [00:00<?, ?it/s]

In [18]:
len(sentences)

47243

In [19]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [20]:
df = pd.DataFrame({'Sentence #' : sentences, 'Word' : texts, 'POS' : poses, 'Tag' : bios})

In [21]:
df.to_csv('bio_data.csv')

In [22]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-Food,1877
1,B-Interior,686
2,B-Price,134
3,B-Service,1246
4,B-Whole,795
5,I-Food,959
6,I-Interior,189
7,I-Price,30
8,I-Service,283
9,I-Whole,187


In [31]:
X = df.drop('Tag', axis=1)
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)
X_train.shape, y_train.shape

((31648, 8324), (31648,))

In [32]:
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(X_train, y_train, classes)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


-- Epoch 1
-- Epoch 1
Norm: 42.10, NNZs: 806, Bias: -4.000000, T: 31648, Avg. loss: 0.058582
Total training time: 0.57 seconds.
-- Epoch 1
Norm: 26.70, NNZs: 324, Bias: -3.000000, T: 31648, Avg. loss: 0.018105
Total training time: 0.58 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.6s


Norm: 13.78, NNZs: 79, Bias: -2.000000, T: 31648, Avg. loss: 0.003887
Total training time: 0.51 seconds.
-- Epoch 1
Norm: 35.62, NNZs: 551, Bias: -3.000000, T: 31648, Avg. loss: 0.037254
Total training time: 0.54 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    1.1s remaining:    0.6s


Norm: 25.48, NNZs: 251, Bias: -3.000000, T: 31648, Avg. loss: 0.020001
Total training time: 0.53 seconds.
Norm: 60.29, NNZs: 1630, Bias: 5.000000, T: 31648, Avg. loss: 0.134985
Total training time: 0.57 seconds.


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    1.7s finished


Perceptron(max_iter=5, n_jobs=-1, verbose=10)

In [33]:
new_classes = classes.copy()
new_classes.pop()
new_classes

['B-Food', 'B-Interior', 'B-Price', 'B-Service', 'B-Whole']

In [34]:
print(classification_report(y_pred=per.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

      B-Food       0.72      0.48      0.57       484
  B-Interior       0.64      0.65      0.64       182
     B-Price       0.69      0.55      0.61        33
   B-Service       0.82      0.60      0.69       359
     B-Whole       0.85      0.37      0.52       216

   micro avg       0.74      0.52      0.61      1274
   macro avg       0.74      0.53      0.61      1274
weighted avg       0.76      0.52      0.61      1274



In [35]:
sgd = SGDClassifier()
sgd.partial_fit(X_train, y_train, classes)

SGDClassifier()

In [36]:
print(classification_report(y_pred=sgd.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

      B-Food       0.90      0.15      0.26       484
  B-Interior       0.87      0.42      0.57       182
     B-Price       0.89      0.24      0.38        33
   B-Service       0.95      0.41      0.57       359
     B-Whole       0.85      0.37      0.51       216

   micro avg       0.90      0.30      0.45      1274
   macro avg       0.89      0.32      0.46      1274
weighted avg       0.90      0.30      0.44      1274



In [37]:
nb = MultinomialNB(alpha=0.01)
nb.partial_fit(X_train, y_train, classes)

MultinomialNB(alpha=0.01)

In [38]:
print(classification_report(y_pred=nb.predict(X_test), y_true=y_test, labels = new_classes))

              precision    recall  f1-score   support

      B-Food       0.33      0.83      0.47       484
  B-Interior       0.70      0.73      0.72       182
     B-Price       0.57      0.79      0.66        33
   B-Service       0.31      0.77      0.44       359
     B-Whole       0.55      0.87      0.67       216

   micro avg       0.38      0.81      0.51      1274
   macro avg       0.49      0.80      0.59      1274
weighted avg       0.42      0.81      0.53      1274



In [39]:
pa =PassiveAggressiveClassifier()
pa.partial_fit(X_train, y_train, classes)

PassiveAggressiveClassifier()

In [40]:
print(classification_report(y_pred=pa.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

      B-Food       0.67      0.63      0.65       484
  B-Interior       0.80      0.65      0.72       182
     B-Price       0.25      0.79      0.38        33
   B-Service       0.74      0.65      0.70       359
     B-Whole       0.73      0.66      0.70       216

   micro avg       0.68      0.65      0.66      1274
   macro avg       0.64      0.68      0.63      1274
weighted avg       0.71      0.65      0.67      1274



In [41]:
! pip install sklearn_crfsuite

     |████████████████████████████████| 743 kB 9.4 MB/s 


In [42]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [43]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
            
getter = SentenceGetter(df)
sentences = getter.sentences

In [44]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features
    
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [52]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.43, random_state=0, shuffle=True)

In [53]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [54]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

              precision    recall  f1-score   support

      B-Food       1.00      1.00      1.00         1
  B-Interior       0.00      0.00      0.00         0
     B-Price       0.00      0.00      0.00         0
   B-Service       0.00      0.00      0.00         0
     B-Whole       1.00      1.00      1.00         3

   micro avg       1.00      1.00      1.00         4
   macro avg       0.40      0.40      0.40         4
weighted avg       1.00      1.00      1.00         4



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-Food', 'B-Interior', 'B-Price', 'B-Service', 'B-Whole'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-Food,1393
1,B-Interior,549
2,B-Price,118
3,B-Service,1031
4,B-Whole,686
5,O,43460


In [55]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
8.030696 B-Service word.lower():обслуживание
5.707952 B-Food   word.lower():поужинать
5.656241 B-Service word.lower():обслуживанием
5.512313 B-Food   word.lower():ужинали
4.994573 O        -1:word.lower():живая
4.631689 B-Service word.lower():обслуживании
4.475218 B-Price  word.lower():цены
4.424017 B-Food   word.lower():пили
4.401647 B-Service word.lower():официантки
4.360559 B-Service word.lower():обслуживала
4.313866 B-Service word.lower():сотрудники
4.297000 B-Interior word.lower():потанцевать
4.261691 B-Food   word.lower():мороженого
4.228068 B-Service word.lower():обслуживания
4.201276 B-Service word.lower():сервисом
4.200410 B-Food   word.lower():шампанское
4.133865 B-Food   word.lower():порции
4.068452 B-Whole  word.lower():заведение
3.793095 B-Price  word.lower():ценник
3.793066 O        -1:word.lower():обслуживающий
3.792899 B-Service word.lower():официантов
3.780186 B-Food   word.lower():приготовлено
3.741545 B-Service word.lower():администратора
3.678601 B-Who